In [110]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import csv
import os

# training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [111]:

DATA_PATH = os.path.join('MP_Data')  # Path for exported data, numpy arrays


#getting dataset values from csv
with open('newlabelstrain.csv', newline='') as f:
    reader = csv.reader(f)
    data = [row[0] for row in reader]


actions = np.array(data)
no_sequences = 30  # thirty videos worth of data
sequence_length = 30  # each video is 30 frame in length

label_map = {label:num for num, label in enumerate(actions)}
print(label_map)


{'niceto': 0, 'yo': 1, 'meetyou': 2, 'thumbsup': 3, 'hi': 4}


In [112]:

sequences, labels = [], []  # x data, y data
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

x = np.array(sequences)
print(x.shape)
y = to_categorical(labels).astype(int)
print(y)


(150, 30, 1662)
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0

In [113]:

# creating training partitions
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(142, 30, 1662) (8, 30, 1662) (142, 5) (8, 5)


In [114]:

# training

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential() # instantiating the model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [115]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])
print(model.summary())

Epoch 1/2000
5/5 [==============================] - 3s 167ms/step - loss: 1.6931 - categorical_accuracy: 0.2254
Epoch 2/2000
5/5 [==============================] - 0s 60ms/step - loss: 1.6363 - categorical_accuracy: 0.2042
Epoch 3/2000
5/5 [==============================] - 0s 61ms/step - loss: 1.5970 - categorical_accuracy: 0.2042
Epoch 4/2000
5/5 [==============================] - 0s 73ms/step - loss: 1.5732 - categorical_accuracy: 0.2042
Epoch 5/2000
5/5 [==============================] - 0s 71ms/step - loss: 1.5605 - categorical_accuracy: 0.1268
Epoch 6/2000
5/5 [==============================] - 0s 71ms/step - loss: 1.8119 - categorical_accuracy: 0.2113
Epoch 7/2000
5/5 [==============================] - 0s 60ms/step - loss: 1.5356 - categorical_accuracy: 0.2887
Epoch 8/2000
5/5 [==============================] - 0s 61ms/step - loss: 1.5093 - categorical_accuracy: 0.2254
Epoch 9/2000
5/5 [==============================] - 0s 60ms/step - loss: 1.5580 - categorical_accuracy: 0.3732


5/5 [==============================] - 0s 59ms/step - loss: 0.6102 - categorical_accuracy: 0.6549
Epoch 75/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.5900 - categorical_accuracy: 0.7183
Epoch 76/2000
5/5 [==============================] - 0s 66ms/step - loss: 0.6200 - categorical_accuracy: 0.7183
Epoch 77/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.5913 - categorical_accuracy: 0.6761
Epoch 78/2000
5/5 [==============================] - 0s 68ms/step - loss: 0.5559 - categorical_accuracy: 0.7183
Epoch 79/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.5725 - categorical_accuracy: 0.7183
Epoch 80/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.5355 - categorical_accuracy: 0.6972
Epoch 81/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.5415 - categorical_accuracy: 0.7394
Epoch 82/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.5490 - categorical_accuracy: 0.7535
Epoch 

5/5 [==============================] - 0s 58ms/step - loss: 0.3950 - categorical_accuracy: 0.8099
Epoch 147/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.4593 - categorical_accuracy: 0.7746
Epoch 148/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.3398 - categorical_accuracy: 0.8521
Epoch 149/2000
5/5 [==============================] - 0s 58ms/step - loss: 6.3583 - categorical_accuracy: 0.8451
Epoch 150/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.3535 - categorical_accuracy: 0.8380
Epoch 151/2000
5/5 [==============================] - 0s 58ms/step - loss: 0.4785 - categorical_accuracy: 0.7746
Epoch 152/2000
5/5 [==============================] - 0s 59ms/step - loss: 0.4171 - categorical_accuracy: 0.8099
Epoch 153/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.3740 - categorical_accuracy: 0.8521
Epoch 154/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.3656 - categorical_accuracy: 0.859

5/5 [==============================] - 0s 57ms/step - loss: 0.5085 - categorical_accuracy: 0.7535
Epoch 219/2000
5/5 [==============================] - 0s 56ms/step - loss: 0.4052 - categorical_accuracy: 0.8099
Epoch 220/2000
5/5 [==============================] - 0s 54ms/step - loss: 0.3478 - categorical_accuracy: 0.8732
Epoch 221/2000
5/5 [==============================] - 0s 54ms/step - loss: 0.3109 - categorical_accuracy: 0.8944
Epoch 222/2000
5/5 [==============================] - 0s 55ms/step - loss: 0.7172 - categorical_accuracy: 0.7958
Epoch 223/2000
5/5 [==============================] - 0s 55ms/step - loss: 1.6246 - categorical_accuracy: 0.4437
Epoch 224/2000
5/5 [==============================] - 0s 55ms/step - loss: 0.9780 - categorical_accuracy: 0.5704
Epoch 225/2000
5/5 [==============================] - 0s 56ms/step - loss: 0.9878 - categorical_accuracy: 0.4930
Epoch 226/2000
5/5 [==============================] - 0s 54ms/step - loss: 0.8468 - categorical_accuracy: 0.570

5/5 [==============================] - 0s 56ms/step - loss: 0.2398 - categorical_accuracy: 0.8803
Epoch 291/2000
5/5 [==============================] - 0s 56ms/step - loss: 0.3371 - categorical_accuracy: 0.8451
Epoch 292/2000
5/5 [==============================] - 0s 56ms/step - loss: 0.3993 - categorical_accuracy: 0.8380
Epoch 293/2000
5/5 [==============================] - 0s 55ms/step - loss: 0.3987 - categorical_accuracy: 0.8099
Epoch 294/2000
5/5 [==============================] - 0s 56ms/step - loss: 0.3799 - categorical_accuracy: 0.8169
Epoch 295/2000
5/5 [==============================] - 0s 55ms/step - loss: 0.3725 - categorical_accuracy: 0.8239
Epoch 296/2000
5/5 [==============================] - 0s 54ms/step - loss: 0.4720 - categorical_accuracy: 0.7606
Epoch 297/2000
5/5 [==============================] - 0s 54ms/step - loss: 0.3480 - categorical_accuracy: 0.8451
Epoch 298/2000
5/5 [==============================] - 0s 57ms/step - loss: 0.4357 - categorical_accuracy: 0.795

...

5/5 [==============================] - 0s 64ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1655/2000
5/5 [==============================] - 0s 67ms/step - loss: 1.6094 - categorical_accuracy: 0.2042
Epoch 1656/2000
5/5 [==============================] - 0s 67ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1657/2000
5/5 [==============================] - 0s 69ms/step - loss: 1.6095 - categorical_accuracy: 0.2042
Epoch 1658/2000
5/5 [==============================] - 0s 73ms/step - loss: 1.6096 - categorical_accuracy: 0.1972
Epoch 1659/2000
5/5 [==============================] - 0s 70ms/step - loss: 1.6095 - categorical_accuracy: 0.1901
Epoch 1660/2000
5/5 [==============================] - 0s 73ms/step - loss: 1.6097 - categorical_accuracy: 0.1972
Epoch 1661/2000
5/5 [==============================] - 0s 71ms/step - loss: 1.6101 - categorical_accuracy: 0.1972
Epoch 1662/2000
5/5 [==============================] - 0s 66ms/step - loss: 1.6097 - categorical_accurac

5/5 [==============================] - 0s 83ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1727/2000
5/5 [==============================] - 0s 94ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1728/2000
5/5 [==============================] - 0s 82ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1729/2000
5/5 [==============================] - 0s 85ms/step - loss: 1.6095 - categorical_accuracy: 0.2042
Epoch 1730/2000
5/5 [==============================] - 0s 81ms/step - loss: 1.6099 - categorical_accuracy: 0.2042
Epoch 1731/2000
5/5 [==============================] - 0s 88ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1732/2000
5/5 [==============================] - 0s 80ms/step - loss: 1.6102 - categorical_accuracy: 0.2042
Epoch 1733/2000
5/5 [==============================] - 0s 74ms/step - loss: 1.6098 - categorical_accuracy: 0.1761
Epoch 1734/2000
5/5 [==============================] - 0s 76ms/step - loss: 1.6096 - categorical_accurac

5/5 [==============================] - 0s 69ms/step - loss: 1.6102 - categorical_accuracy: 0.2042
Epoch 1799/2000
5/5 [==============================] - 0s 65ms/step - loss: 1.6099 - categorical_accuracy: 0.2042
Epoch 1800/2000
5/5 [==============================] - 0s 66ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1801/2000
5/5 [==============================] - 0s 65ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1802/2000
5/5 [==============================] - 0s 66ms/step - loss: 1.6095 - categorical_accuracy: 0.2042
Epoch 1803/2000
5/5 [==============================] - 0s 65ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1804/2000
5/5 [==============================] - 0s 69ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1805/2000
5/5 [==============================] - 0s 65ms/step - loss: 1.6097 - categorical_accuracy: 0.1901
Epoch 1806/2000
5/5 [==============================] - 0s 68ms/step - loss: 1.6098 - categorical_accurac

5/5 [==============================] - 0s 65ms/step - loss: 1.6099 - categorical_accuracy: 0.2042
Epoch 1871/2000
5/5 [==============================] - 0s 67ms/step - loss: 1.6102 - categorical_accuracy: 0.2042
Epoch 1872/2000
5/5 [==============================] - 0s 65ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1873/2000
5/5 [==============================] - 0s 67ms/step - loss: 1.6095 - categorical_accuracy: 0.2042
Epoch 1874/2000
5/5 [==============================] - 0s 65ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1875/2000
5/5 [==============================] - 0s 64ms/step - loss: 1.6093 - categorical_accuracy: 0.2042
Epoch 1876/2000
5/5 [==============================] - 0s 66ms/step - loss: 1.6100 - categorical_accuracy: 0.2042
Epoch 1877/2000
5/5 [==============================] - 0s 64ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1878/2000
5/5 [==============================] - 0s 67ms/step - loss: 1.6097 - categorical_accurac

Epoch 1942/2000
5/5 [==============================] - 0s 74ms/step - loss: 1.6099 - categorical_accuracy: 0.2042
Epoch 1943/2000
5/5 [==============================] - 0s 73ms/step - loss: 1.6094 - categorical_accuracy: 0.2042
Epoch 1944/2000
5/5 [==============================] - 0s 74ms/step - loss: 1.6096 - categorical_accuracy: 0.2042
Epoch 1945/2000
5/5 [==============================] - 0s 77ms/step - loss: 1.6093 - categorical_accuracy: 0.2042
Epoch 1946/2000
5/5 [==============================] - 0s 87ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1947/2000
5/5 [==============================] - 0s 73ms/step - loss: 1.6095 - categorical_accuracy: 0.2042
Epoch 1948/2000
5/5 [==============================] - 0s 75ms/step - loss: 1.6101 - categorical_accuracy: 0.2042
Epoch 1949/2000
5/5 [==============================] - 0s 72ms/step - loss: 1.6097 - categorical_accuracy: 0.2042
Epoch 1950/2000
5/5 [==============================] - 0s 75ms/step - loss: 1.6097 - cat

In [107]:
res = model.predict(x_test)
print(res)
print(actions[np.argmax(res[0])])
for i in range(len(res)):
    print(actions[np.argmax(res[i])])

[[8.62513592e-14 1.00000000e+00 4.69410336e-29]
 [1.22290401e-24 1.00000000e+00 1.54491207e-15]
 [1.05704596e-13 1.00000000e+00 3.24167683e-29]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [2.39385379e-04 0.00000000e+00 9.99760568e-01]]
meetyou
meetyou
meetyou
meetyou
meetyou
hi


In [108]:
# save weights
model.save('action.h5')

In [109]:
#evaluation using confusion matrix
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
matrix = multilabel_confusion_matrix(ytrue, yhat)
print(matrix)
accuracy_score = accuracy_score(ytrue, yhat)
print(f"{accuracy_score= }")


[[[3 0]
  [2 0]]

 [[1 2]
  [0 2]]

 [[4 0]
  [0 1]]]
accuracy_score= 0.6
